In [3]:
from pathlib import Path
from cloudpathlib import S3Path, S3Client
import nibabel as nib 
import neuropythy as ny
from nilearn import datasets, input_data, connectome
import numpy as np

# Make sure that we have a cache path:
cache_path = Path('/tmp/cache')
if not cache_path.exists():
    cache_path.mkdir()

hcp_base_path = S3Path(
    's3://hcp-openaccess/HCP_1200/',
    client=S3Client(
        local_cache_dir=cache_path,
        profile_name='hcp'))

/Users/jacob/opt/anaconda3/lib/python3.7/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


NameError: name 'Session' is not defined

In [4]:
hcp_subdirs = list(hcp_base_path.iterdir())
# There are about 1200 of these, so we won't show them all, just the first 10:
hcp_subdirs[:10]


NameError: name 'hcp_base_path' is not defined

In [2]:
# Load preprocessed fMRI data
sub115825 = 'shared/HCP/115825/MNINonLinear/Results/rfMRI_REST1_7T_PA/rfMRI_REST1_7T_PA_Atlas_1.6mm_hp2000_clean.dtseries.nii'
subj_bold = ny.load(sub115825)
subj_bold

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [33]:
(lh_data_bold, rh_data_bold, subcortex_data_bold) = ny.hcp.cifti_split(subj_bold)

In [4]:
# BOLD signal for every vertex
lh_data_bold.shape

(900, 59292)

In [5]:
rh_data_bold.shape

(900, 59292)

In [65]:
subcortex_data_bold.shape

(900, 62053)

In [6]:
sub115825_labels = 'shared/HCP/115825/MNINonLinear/fsaverage_LR59k/115825.aparc.59k_fs_LR.dlabel.nii'
subj_labels = nib.load(sub115825_labels)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [32]:
(lh_data_labels, rh_data_labels, subcortex_data_labels) = ny.hcp.cifti_split(subj_labels)

In [10]:
# label ID for every vertex
lh_data_labels.shape

(1, 59292)

In [11]:
rh_data_labels.shape

(1, 59292)

In [ ]:
# go through every unique label 
# for every label find the vertices with that label
# average BOLD signal for all the vertices
## ==> smaller matrix, 900 x number of labels (about 50 maybe)


In [21]:
# merge BOLD signals and labels together
lh_data = np.vstack((lh_data_bold, lh_data_labels))
lh_data.shape

(901, 59292)

In [53]:
# unique lh label
unique_lh = np.unique(lh_data_labels)
unique_lh

array([ 0.,  1.,  2.,  3.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., nan])

In [6]:
### Generate N x t 2D matrix of mean BOLD signal for each subject

avg_lh_bold_mat = []

# loop through unique lh labels
for label in unique_lh[unique_lh != 0]: # for each label that isn't 0
    label_idx = np.where(lh_data_labels == label)[1] # get the index of the vertices that match this label
    label_timeseriesda = np.mean(lh_data_bold[:, label_idx], axis = 1) # take the mean of the vertices within the region
    avg_lh_bold_mat.append(label_timeseriesda) # append the mean BOLD signal of this region at each time point (volume)
avg_lh_bold_mat = np.vstack(avg_lh_bold_mat) # stack all the regions together into a matrix

avg_rh_bold_mat = []

# loop through unique rh labels
for label in unique_rh[unique_rh != 0]: # do the same in the RH
    label_idx = np.where(rh_data_labels == label)[1]
    label_timeseriesda = np.mean(rh_data_bold[:, label_idx], axis = 1)
    avg_rh_bold_mat.append(label_timeseriesda)  
avg_rh_bold_mat = np.vstack(avg_rh_bold_mat) 

avg_bold_mat = np.vstack((avg_lh_bold_mat, avg_rh_bold_mat)) # stack the LH and RH matrices together

NameError: name 'unique_lh' is not defined

In [ ]:
### Generate N x N FC matrix for each subject
subj_bold_data = subj_bold.get_fdata()

num_regions = avg_bold_mat.shape[0]
FC_mat = np.zeros((num_regions, num_regions)) # intialize empty matrix

# loop through each region index
for i_region in range(num_regions): # for each region (x-axis)
    for j_region in range(num_regions): # and for each region (y-axis)
        if i_region == j_region: 
            FC_mat == 0 # the correlation between the region and itself is set to 0 by convention
        else: # otherwise compute the pearson correlation between the time series of each region with every other region
            FC_mat[i, j] = pearsonr(subj_bold_data[:, i_region], subj_bold_data[:, j_region])[0]

FC_mat_fisherz = np.arctanh(FC_mat) # to make the distribution of the values more normal, fisher-z transform them

In [50]:
rh_data = np.vstack((rh_data_bold, rh_data_labels))
rh_data.shape

(901, 59292)

In [52]:
unique_rh = np.unique(rh_data_labels)
unique_rh

array([ 0., 36., 37., 38., 40., 41., 42., 43., 44., 45., 46., 47., 48.,
       49., 50., 51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61.,
       62., 63., 64., 65., 66., 67., 68., 69., 70., nan])

In [45]:
# merge labels together
all_data_labels = np.hstack((lh_data_labels, rh_data_labels))
all_data_labels


900

In [30]:
# get unique labels
unique_labels = np.unique(all_data_labels)
unique_labels

array([ 0.,  1.,  2.,  3.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 40.,
       41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53.,
       54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65., 66.,
       67., 68., 69., 70., nan])

In [48]:
FC_matrix = np.zeros((len(all_data_bold),len(unique_labels)))
average_bold_list = []

for label in unique_labels:
    # Find vertices with the current label
    label_vertices = np.where(all_data_labels == label)[0]
    
    # Get the BOLD signal for vertices with the current label
    label_bold = all_data_bold[label_vertices]
    
    # Calculate the average BOLD signal along the first axis (vertices)
    average_bold = np.mean(label_bold, axis=0)
    
    average_bold_list.append(average_bold)
    
FC_matrix = np.array(average_bold_list).T

In [49]:
FC_matrix.shape

(118584, 70)

AttributeError: module 'numpy' has no attribute 'which'

In [27]:
label_vertices.shape

(0,)

In [14]:
avg_bold_label = []

## loop through unique labels
for label in unique_labels:
    label_vertices = np.where(all_data_labels == label)[0] # identify the vertices with label
    label_bold = all_data_bold[label_vertices] # extract BOLD for the vertices
    avg_bold = np.mean(label_bold, axis=0) # average BOLD 
    avg_bold_label.append(avg_bold) 
    
avg_bold_label = np.array(avg_bold_label) # each row corresponds to the avg BOLD for each unique label 

/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [23]:
avg_bold_label

array([[11089.08984375, 12419.76953125, 13332.8515625 , ...,
        12756.04003906, 11879.54101562, 12271.46972656],
       [11089.08984375, 12419.76953125, 13332.8515625 , ...,
        12756.04003906, 11879.54101562, 12271.46972656],
       [11089.08984375, 12419.76953125, 13332.8515625 , ...,
        12756.04003906, 11879.54101562, 12271.46972656],
       ...,
       [11089.08984375, 12419.76953125, 13332.8515625 , ...,
        12756.04003906, 11879.54101562, 12271.46972656],
       [11089.08984375, 12419.76953125, 13332.8515625 , ...,
        12756.04003906, 11879.54101562, 12271.46972656],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan]])

In [ ]:
## Ignore from here (for now)

In [21]:
# Choose Atlas
atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
# atlas = sub115825.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')

In [22]:
# Create Masking Object
masker = input_data.NiftiLabelsMasker(labels_img=atlas.maps, standardize=True)

In [23]:
# Extract time-series for each ROIs
time_series = masker.fit_transform(sub115825)

/srv/conda/envs/notebook/lib/python3.10/site-packages/joblib/memory.py:353: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  return self.func(*args, **kwargs)


In [25]:
# Correlation FC matrix
correlation_matrix = connectome.ConnectivityMeasure(kind='correlation').fit_transform([time_series])[0]
print(correlation_matrix)

[[1.         0.8119864  0.9464104  ... 0.7323204  0.611148   0.5107898 ]
 [0.8119864  1.         0.8052994  ... 0.8699405  0.6637082  0.56077397]
 [0.9464104  0.8052994  1.         ... 0.7288159  0.63289195 0.47462818]
 ...
 [0.7323204  0.8699405  0.7288159  ... 1.         0.68419755 0.48319957]
 [0.611148   0.6637082  0.63289195 ... 0.68419755 1.         0.6632808 ]
 [0.5107898  0.56077397 0.47462818 ... 0.48319957 0.6632808  1.        ]]


/srv/conda/envs/notebook/lib/python3.10/site-packages/nilearn/connectome/connectivity_matrices.py:495: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [


In [10]:
sub100610_freesurfer_path = hcp_base_path / '100610' / 'T1w' / '100610'


In [11]:
# If we want to load one of these files into nibabel, we can convert it
# into a filesystem path (cloudpaths cannot just be passed to nibabel).

import nibabel as nib

brain_img_path = sub100610_freesurfer_path / 'mri' / 'brain.mgz'
brain_img = nib.load(brain_img_path.fspath)

In [14]:
# Selecting ROIs using masks

import numpy as np

msdl_atlas = hcp_base_path.fetch_atlas_msdl()

msdl_coords = msdl_atlas.region_coords
n_regions = len(msdl_coords)

print(f'MSDL has {n_regions} ROIs, part of the following networks :\n{np.unique(msdl_atlas.networks)}.')

AttributeError: 'S3Path' object has no attribute 'fetch_atlas_msdl'